In [1]:
import pandas as pd


df = pd.read_csv('Loan.csv')


print(df.head())



    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001015   Male     Yes          0      Graduate            No   
1  LP001022   Male     Yes          1      Graduate            No   
2  LP001031   Male     Yes          2      Graduate            No   
3  LP001035   Male     Yes          2      Graduate            No   
4  LP001051   Male      No          0  Not Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History Property_Area  
0             1.0         Urban  
1             1.0         Urban  
2             1.0         Urban  
3             NaN     

In [2]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB
None


In [3]:
print(df.describe(include='all'))

         Loan_ID Gender Married Dependents Education Self_Employed  \
count        367    356     367        357       367           344   
unique       367      2       2          4         2             2   
top     LP001015   Male     Yes          0  Graduate            No   
freq           1    286     233        200       283           307   
mean         NaN    NaN     NaN        NaN       NaN           NaN   
std          NaN    NaN     NaN        NaN       NaN           NaN   
min          NaN    NaN     NaN        NaN       NaN           NaN   
25%          NaN    NaN     NaN        NaN       NaN           NaN   
50%          NaN    NaN     NaN        NaN       NaN           NaN   
75%          NaN    NaN     NaN        NaN       NaN           NaN   
max          NaN    NaN     NaN        NaN       NaN           NaN   

        ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count        367.000000         367.000000  362.000000        361.000000   
unique 

In [4]:

missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)


df.fillna(df.mean(numeric_only=True), inplace=True)
df.fillna(df.mode().iloc[0], inplace=True)
removed_missing_values=df.isnull().sum()


Missing Values:
 Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64


In [5]:

duplicates = df.duplicated().sum()
print(f"Duplicate Records: {duplicates}")

df.drop_duplicates(inplace=True)


Duplicate Records: 0


In [6]:

for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = pd.to_datetime(df[col])
            print(f"Converted {col} to datetime.")
        except:
            try:
                df[col] = pd.to_numeric(df[col])
                print(f"Converted {col} to numeric.")
            except:
                continue 


C:\Users\admin\AppData\Local\Temp\ipykernel_19536\3456982513.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
C:\Users\admin\AppData\Local\Temp\ipykernel_19536\3456982513.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
C:\Users\admin\AppData\Local\Temp\ipykernel_19536\3456982513.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])
C:\Users\admin\AppData\Local\Temp\ipykernel_19536\3456982513.py:4: UserWarning: Could not infer format, so each element will be parsed indi

In [7]:

numeric_cols = df.select_dtypes(include=['number'])


Q1 = numeric_cols.quantile(0.25)
Q3 = numeric_cols.quantile(0.75)
IQR = Q3 - Q1


non_outlier_mask = ~((numeric_cols < (Q1 - 1.5 * IQR)) | (numeric_cols > (Q3 + 1.5 * IQR))).any(axis=1)


df = df[non_outlier_mask]


df.reset_index(drop=True, inplace=True)

print(df.shape)

(210, 12)


In [8]:
from sklearn.preprocessing import StandardScaler

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns


scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [9]:

for col in numeric_cols:
    if (df[col] < 0).any():
        print(f"Warning: Negative values found in {col}")
     
        df = df[df[col] >= 0]


#Summary of Cleaning Steps Performed
Loaded the dataset and inspected structure and summary stats.

Identified and handled missing values using mean/mode imputation.

Removed duplicate records.

Converted object-type columns to datetime or numeric if applicable.

Detected and removed outliers using IQR method.

Standardized all numerical features using StandardScaler.

Performed basic validation for negative values in numerical fields.